In [ ]:
from neuprint import Client
from neuprint import fetch_neurons
from neuprint import NeuronCriteria as NC
from neuprint import SynapseCriteria as SC
import bokeh.palettes
from bokeh.models import Range1d
from bokeh.plotting import figure, show, output_notebook
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from neuprint import fetch_synapses, fetch_adjacencies
from neuprint import SynapseCriteria as SC
output_notebook()


with open('auth_token.txt', 'r') as f:
    token = f.read()

c = Client('neuprint.janelia.org', dataset='manc:v1.0', token=token)
c.fetch_version()

import pandas as pd

from matplotlib import rcParams

rcParams['font.family'] = 'Arial'
rcParams['font.size'] = 9

cluster2 = [14640, 34562, 16090, 18039, 101455, 16410, 101860]


In [ ]:
CINNABAR = '#db544b'
LAPIS = '#2d66a5'
BYZANTINE = '#D342BE'
SHAMROCK = '#33A358'

roi_mesh = c.fetch_roi_mesh('ANm')

from bokeh.plotting import figure, show, output_notebook
output_notebook()
def plot_neurons(
        source_df, idx : int,
        flattened_ax : int = 1,
        on_p = None,
        weight : float = 1,
        color : str = '#000000',
        plot_synapses : bool = False,
        roi_mesh = c.fetch_roi_mesh('ANm'),
        roi_mesh_alpha : float = 0.5
        ):

        skelee = c.fetch_skeleton(source_df.iloc[idx]['bodyId'])

        skelee.radius = skelee.radius.astype(float)/(20)
        # Join parent/child nodes for plotting as line segments below.
        # (Using each row's 'link' (parent) ID, find the row with matching rowId.)
        skelee = skelee.merge(skelee, 'inner',
                                left_on=['link'],
                                right_on=['rowId'],
                                suffixes=['_child', '_parent'])
        if on_p is None:
            p = figure(
                match_aspect = True
            )
            p.y_range.flipped = False
        else:
            p = on_p

        p.toolbar.logo = None
        p.toolbar_location = None

        axes = [x for x in range(3) if x != flattened_ax]
        if not roi_mesh is None:
            if not isinstance(roi_mesh, list):
                verts = np.array([
                    [float(row.split(' ')[axes[0]+1]),float(row.split(' ')[axes[1]+1])]
                    for row in str(roi_mesh).split('\\n') if row.startswith('v')
                ])
            else:
                verts = np.array([
                    [float(row.split(' ')[axes[0]+1]),float(row.split(' ')[axes[1]+1])]
                    for row in str(roi_mesh[0]).split('\\n') if row.startswith('v')
                ])
                for mesh in roi_mesh:
                    verts = np.concatenate((
                        verts,
                        np.array([
                            [float(row.split(' ')[axes[0]+1]),float(row.split(' ')[axes[1]+1])]
                            for row in str(mesh).split('\\n') if row.startswith('v')
                        ])
                    ))

            speckle_size = 1.0
            p.scatter(verts[:,0],verts[:,1],size=speckle_size,line_alpha=0.0, fill_color = (0,0,0), fill_alpha = roi_mesh_alpha)

        str_labels = ['x', 'y', 'z']

        as_str = [str_labels[i] for i in axes]

        # Plot skeleton segments (in 2D)
        p.segment(
            x0=f'{as_str[0]}_child', x1=f'{as_str[0]}_parent',
            y0=f'{as_str[1]}_child', y1=f'{as_str[1]}_parent',
            line_width = 'radius_parent',
            alpha = 0.7*weight,
            color = color,
            source=skelee
        )

        if plot_synapses:
            syns = fetch_synapses(NC(bodyId=source_df.iloc[idx]['bodyId']))
            pre = syns.loc[syns['type'] == 'pre']
            p.scatter(
                pre[as_str[0]], pre[as_str[1]],
                size = 3, fill_color=BYZANTINE,
                fill_alpha = 0.9, line_color = None,
                line_alpha = 0.0, line_width=0,
            )
            
            post = syns.loc[syns['type'] == 'post']
            
            p.scatter(
                post[as_str[0]], post[as_str[1]],
                size = 3, fill_color=SHAMROCK,
                fill_alpha = 0.9, line_color=None,
                line_width = 0,
            )        

        p.renderers.insert(0,p.renderers.pop())

        if not (roi_mesh is None or isinstance(roi_mesh, list)):
            lims = {
                'x' : Range1d(18000,30000),
                'y' : Range1d(0,20000),
                'z' : Range1d(0,20000)
            }

            if as_str[0] in lims:
                p.x_range = lims[as_str[0]]
            if as_str[1] in lims:
                p.y_range = lims[as_str[1]]

        #p.y_range = Range1d(0,20000)
        #p.x_range = Range1d(18000,30000)
        p.xgrid.visible = False
        p.ygrid.visible = False
        p.axis.visible = False
        return p

from bokeh.plotting import gridplot

from bokeh.io import export_png

In [ ]:
_, upstream_c2 = fetch_adjacencies(
    targets = NC(
        bodyId = cluster2
    ),
)

upstream_c2_grouped = upstream_c2.groupby(
    'bodyId_pre'
).agg(
    {
        'weight' : 'sum'
    }
).sort_values(
    by='weight',
    ascending=False
).reset_index().set_index('bodyId_pre')

upstream_c2_grouped['numerical_idx'] = range(len(upstream_c2_grouped))


cdn_in, _ = fetch_neurons(
    NC(
        bodyId = upstream_c2_grouped.index.values
    )
)

cdn_in.set_index('bodyId', inplace=True)

In [ ]:
cdn_in[cdn_in['entryNerve'].apply(lambda x: 'AbNT' in x if not (x is None) else False)]

In [ ]:
upstream_c2_grouped['output_type'] = 'unknown'

upstream_c2_grouped['output_type'][
    cdn_in[cdn_in['entryNerve'].apply(lambda x: 'AbNT' in x if not (x is None) else False)].index.values
] = 'ascending'

upstream_c2_grouped['output_type'][
    cdn_in[cdn_in['entryNerve'].apply(lambda x: 'CvC' in x if not (x is None) else False)].index.values
] = 'descending'

upstream_c2_grouped['output_type'][
    cdn_in[cdn_in['outputRois'].apply(lambda x: len(x) > 1) * cdn_in['entryNerve'].apply(lambda x : x is None)].index.values
] = 'interneuropil'

upstream_c2_grouped['output_type'][
    cdn_in[cdn_in['outputRois'].apply(lambda x: x == ['ANm']) * cdn_in['entryNerve'].apply(lambda x : x is None)].index.values
] = 'interneuron'

In [ ]:
all_upstream_synapses_f, all_upstream_synapses_x = plt.subplots(nrows = 1, ncols = 1, figsize=(9,1.5))

all_upstream_synapses_x.plot(
    upstream_c2_grouped[upstream_c2_grouped['output_type'] == 'unknown']['numerical_idx'].values,
    upstream_c2_grouped[upstream_c2_grouped['output_type'] == 'unknown']['weight'].values,
    'ok'
)

all_upstream_synapses_x.plot(
    upstream_c2_grouped[upstream_c2_grouped['output_type'] == 'interneuron']['numerical_idx'],
    upstream_c2_grouped[upstream_c2_grouped['output_type'] == 'interneuron']['weight'].values,
    color = '#58479E',
    marker = 'o',
    linewidth = 0,
    alpha = 0.9
)

all_upstream_synapses_x.plot(
    upstream_c2_grouped[upstream_c2_grouped['output_type'] == 'ascending']['numerical_idx'],
    upstream_c2_grouped[upstream_c2_grouped['output_type'] == 'ascending']['weight'].values,
    color = BYZANTINE,
    marker = 'o',
    linewidth = 0,
    alpha = 0.9
)

all_upstream_synapses_x.plot(
    upstream_c2_grouped[upstream_c2_grouped['output_type'] == 'interneuropil']['numerical_idx'],
    upstream_c2_grouped[upstream_c2_grouped['output_type'] == 'interneuropil']['weight'].values,
    color = SHAMROCK,
    marker = 'o',
    linewidth = 0,
    alpha = 0.9,
)

all_upstream_synapses_x.plot(
    upstream_c2_grouped[upstream_c2_grouped['output_type'] == 'descending']['numerical_idx'],
    upstream_c2_grouped[upstream_c2_grouped['output_type'] == 'descending']['weight'].values,
    color = LAPIS,
    marker = 'o',
    linewidth = 0,
    alpha = 0.9
)



all_upstream_synapses_x.set_ylabel('Synapses onto CDNs')
all_upstream_synapses_x.set_xlabel('Presynaptic cells')
#all_upstream_synapses_x.set_xticks([])
all_upstream_synapses_x.spines['right'].set_visible(False)
all_upstream_synapses_x.spines['top'].set_visible(False)
all_upstream_synapses_x.spines['bottom'].set_visible(False)

#all_upstream_synapses_f.savefig('CDN_cluster2_upstream_synapse_count_by_category.svg')

all_upstream_synapses_x.set_xlim(-1, 150)
all_upstream_synapses_f.savefig('CDN_inputs_by_category_zoom.svg')


In [ ]:
upstream_c2.pivot_table(index='bodyId_post', columns='bodyId_pre', values='weight', aggfunc='sum', fill_value=0)[upstream_c2_grouped.index.values]

sns.heatmap(
    upstream_c2.pivot_table(index='bodyId_post', columns='bodyId_pre', values='weight', aggfunc='sum', fill_value=0)[upstream_c2_grouped.index.values[:150]],
    cmap = 'Blues',
    cbar = False,
    xticklabels = False,
    yticklabels = False,
    vmin = 0,
    vmax = 80,
)

plt.gcf().set_size_inches(7, 1)
plt.savefig('CDN_inputs_individual_neurons_heatmap.svg')

In [ ]:
# Plot all descending neurons
internp = cdn_in[upstream_c2_grouped['output_type'] == 'descending']
internp['weight'] = upstream_c2_grouped.loc[internp.index.values]['weight']
internp['bodyId'] = internp.index.values
internp.sort_values(by='weight', ascending=False, inplace=True)

max_val = internp['weight'].max()

all_rois = [c.fetch_roi_mesh(roiname) for roiname in ['ANm', 'LegNp(T1)(L)',
 'LegNp(T1)(R)',
 'LegNp(T2)(L)',
 'LegNp(T2)(R)',
 'LegNp(T3)(L)',
 'LegNp(T3)(R)',]
]

# make a grid
on_ps = [
     plot_neurons(internp, 0, 0, color = LAPIS, weight = internp['weight'].iloc[0]/max_val, roi_mesh=all_rois),
     plot_neurons(internp, 0, 1, color = LAPIS, weight = internp['weight'].iloc[0]/max_val, roi_mesh=all_rois),
     plot_neurons(internp, 0, 2, color = LAPIS, weight = internp['weight'].iloc[0]/max_val, roi_mesh=all_rois),
]
for idx in range(1, 51):
     plot_neurons(internp, idx, 0, on_p = on_ps[0], color = LAPIS, weight = internp['weight'].iloc[idx]/max_val, roi_mesh=None) 
     plot_neurons(internp, idx, 1, on_p = on_ps[1], color = LAPIS, weight = internp['weight'].iloc[idx]/max_val, roi_mesh = None)
     plot_neurons(internp, idx, 2, on_p = on_ps[2], color = LAPIS, weight = internp['weight'].iloc[idx]/max_val, roi_mesh = None)

grid_time = gridplot([on_ps])

#show(grid_time)

show(grid_time)

export_png(grid_time, filename = 'CDN_descending_inputs.png')


In [ ]:
descending_by_roi = fetch_neurons(
        NC(bodyId = upstream_c2_grouped[upstream_c2_grouped['output_type'] == 'descending'].index.values[:150])
    )[1].pivot_table(
        index = 'bodyId',
        columns = 'roi',
        values = 'pre',
)

sns.heatmap(
    descending_by_roi[['ANm', 'LegNp(T1)(L)', 'LegNp(T1)(R)', 'LegNp(T2)(L)', 'LegNp(T2)(R)', 'LegNp(T3)(L)', 'LegNp(T3)(R)', 'IntTct', 'WTct(UTct-T2)(L)', 'WTct(UTct-T2)(R)',]],
    cmap = 'Blues',
    cbar = False,
    xticklabels = True,
    yticklabels = False,
    vmin = 0,
    vmax = 200,
)

plt.gcf().set_size_inches(2, 2)
#plt.savefig('CDN_cluster2_descending_inputs_by_target_roi.svg')

In [ ]:
_, descending_to_cdn = fetch_adjacencies(
    sources = NC(bodyId = upstream_c2_grouped[upstream_c2_grouped['output_type'] == 'descending'].index.values[:150]),
    targets = cluster2,
)

cdn_synapse_count = descending_to_cdn.groupby(
    'bodyId_pre'
).agg(
    {
        'weight' : 'sum'
    }
).sort_values(
    by='weight',
    ascending=False
).reset_index().set_index('bodyId_pre')

cdn_prop_df = cdn_synapse_count.merge(descending_by_roi['ANm'], left_on='bodyId_pre', right_index=True)

plt.plot(
    cdn_prop_df['weight'],
    cdn_prop_df['weight']/cdn_prop_df['ANm'],
    'ok'
)

plt.gca().set_xlabel('Synapses onto CDN')
plt.gca().set_ylabel('Fraction of ANm\nsynapses onto CDNs')

plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

plt.gca().set_yticks([0, 0.25, 0.5])

plt.gcf().set_size_inches(2,2)

plt.savefig('CDN_descending_inputs_fraction_on_cdns.svg')

In [ ]:
# Plot all interneuropil neurons
internp = cdn_in[upstream_c2_grouped['output_type'] == 'interneuropil']
internp['weight'] = upstream_c2_grouped.loc[internp.index.values]['weight']
internp['bodyId'] = internp.index.values
internp.sort_values(by='weight', ascending=False, inplace=True)

max_val = internp['weight'].max()

all_rois = [c.fetch_roi_mesh(roiname) for roiname in ['ANm', 'LegNp(T1)(L)',
 'LegNp(T1)(R)',
 'LegNp(T2)(L)',
 'LegNp(T2)(R)',
 'LegNp(T3)(L)',
 'LegNp(T3)(R)',]
]

# make a grid
on_ps = [
     plot_neurons(internp, 0, 0, color = SHAMROCK, weight = internp['weight'].iloc[0]/max_val, roi_mesh=all_rois),
     plot_neurons(internp, 0, 1, color = SHAMROCK, weight = internp['weight'].iloc[0]/max_val, roi_mesh=all_rois),
     plot_neurons(internp, 0, 2, color = SHAMROCK, weight = internp['weight'].iloc[0]/max_val, roi_mesh=all_rois),
]
for idx in range(1, 51):
     plot_neurons(internp, idx, 0, on_p = on_ps[0], color = SHAMROCK, weight = internp['weight'].iloc[idx]/max_val, roi_mesh=None) 
     plot_neurons(internp, idx, 1, on_p = on_ps[1], color = SHAMROCK, weight = internp['weight'].iloc[idx]/max_val, roi_mesh = None)
     plot_neurons(internp, idx, 2, on_p = on_ps[2], color = SHAMROCK, weight = internp['weight'].iloc[idx]/max_val, roi_mesh = None)

grid_time = gridplot([on_ps])

#show(grid_time)

show(grid_time)

export_png(grid_time, filename = 'CDN_interneuropil_inputs.png')


In [ ]:
internp_by_roi = fetch_neurons(
        NC(bodyId = upstream_c2_grouped[upstream_c2_grouped['output_type'] == 'interneuropil'].index.values[:150])
    )[1].pivot_table(
        index = 'bodyId',
        columns = 'roi',
        values = 'pre',
)

sns.heatmap(
    internp_by_roi[['ANm', 'LegNp(T1)(L)', 'LegNp(T1)(R)', 'LegNp(T2)(L)', 'LegNp(T2)(R)', 'LegNp(T3)(L)', 'LegNp(T3)(R)', 'IntTct', 'WTct(UTct-T2)(L)', 'WTct(UTct-T2)(R)',]],
    cmap = 'Greens',
    cbar = False,
    xticklabels = True,
    yticklabels = False,
    vmin = 0,
    vmax = 400,
)

plt.gcf().set_size_inches(2, 2)
plt.savefig('CDN_interneuropil_inputs_by_target_roi.svg')

In [ ]:
_, interneuropil_to_cdn = fetch_adjacencies(
    sources = NC(bodyId = upstream_c2_grouped[upstream_c2_grouped['output_type'] == 'interneuropil'].index.values[:150]),
    targets = cluster2,
)

cdn_synapse_count = interneuropil_to_cdn.groupby(
    'bodyId_pre'
).agg(
    {
        'weight' : 'sum'
    }
).sort_values(
    by='weight',
    ascending=False
).reset_index().set_index('bodyId_pre')

summed = internp_by_roi.sum(axis=1)
summed.name = 'summed'

cdn_prop_df = cdn_synapse_count.merge(internp_by_roi['ANm'], left_on='bodyId_pre', right_index=True)
cdn_prop_df = cdn_prop_df.merge(summed, left_on='bodyId_pre', right_index=True)

plt.plot(
    cdn_prop_df['summed'],
    cdn_prop_df['weight'],
    'o',
    color = SHAMROCK,
)

plt.gca().set_ylabel('Synapses onto CDNs')
plt.gca().set_xlabel('Total release sites')

plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

#plt.gca().set_yticks([0, 0.5, 1.0])
plt.plot([0, 1000], [0, 1000], '--k')

plt.gca().set_ylim(0,420)
plt.gca().set_yticks([0, 200, 400])

plt.gcf().set_size_inches(2,1)

plt.savefig('CDN_interneuropil_inputs_fraction_on_cdns.svg')

In [ ]:
top_inputs_by_cdn = interneuropil_to_cdn.pivot_table(index='bodyId_pre', columns='bodyId_post', values='weight', aggfunc='sum', fill_value=0).loc[cdn_prop_df.index.values[:4]]

sns.heatmap(
    top_inputs_by_cdn,
    cmap = 'Greens',
    cbar = False,
    xticklabels = False,
    yticklabels = True,
    vmin = 0,
    vmax = 100,
)

plt.yticks(rotation=0)
plt.gcf().set_size_inches(2, 1)

plt.gcf().savefig('CDN_interneuropil_inputs_to_each_cdn_heatmap.svg')

In [ ]:
top_internp_neurons = cdn_prop_df.iloc[:4]
top_internp_neurons['bodyId'] = top_internp_neurons.index.values

from matplotlib.patches import Rectangle

for cell_id in top_internp_neurons.index.values:
    inputs_df, weights_df = fetch_adjacencies(targets=NC(bodyId=cell_id))
    inputs_df = inputs_df.merge(weights_df, left_on='bodyId', right_on='bodyId_pre')
    inputs_df['input_type'] = 'unknown'

    inputs_df['input_type'][
        inputs_df[inputs_df['type'].apply(lambda x: 'DN' in x if x is not None else False)].index.values
    ] = 'descending'

    inputs_df['input_type'][
        inputs_df[inputs_df['type'].apply(lambda x: 'IN' in x if x is not None else False)].index.values
    ] = 'interneuron'

    inputs_df['input_type'][
        inputs_df[inputs_df['type'].apply(lambda x: 'SN' in x if x is not None else False)].index.values
    ] = 'sensory' 

    bar_plot_f, bar_plot_x = plt.subplots(1,1, figsize=(0.5,2))

    bar_plot_x.set_ylim(0, 1)
    bar_plot_x.set_xlim(0, 1)

    print(f"Cell ID: {cell_id}")
    y = 0

    width = inputs_df['input_type'][inputs_df['input_type'] == 'interneuron'].count()/inputs_df['input_type'].count()
    print(f"Interneuron: {width}")

    bar_plot_x.add_patch(
        Rectangle((0,y), 1, width, color = '#58479E')
    )
    y+= width

    width = inputs_df['input_type'][inputs_df['input_type'] == 'descending'].count()/inputs_df['input_type'].count()
    print(f"Descending: {width}")
    
    bar_plot_x.add_patch(
        Rectangle((0,y), 1, width, color = LAPIS)
    )

    y+= width

    width = inputs_df['input_type'][inputs_df['input_type'] == 'sensory'].count()/inputs_df['input_type'].count()
    print(f"Sensory: {width}")
    
    bar_plot_x.add_patch(
        Rectangle((0,y), 1, width, color = BYZANTINE)
    )

    y+= width

    bar_plot_x.add_patch(
        Rectangle((0,y), 1, 1-y, color = '#000000', fill = False, linewidth = 2)
    )
    

    bar_plot_x.set_xticks([])
    bar_plot_x.set_yticks([])

    bar_plot_x.spines['right'].set_visible(False)
    bar_plot_x.spines['top'].set_visible(False)
    bar_plot_x.spines['bottom'].set_visible(False)
    bar_plot_x.spines['left'].set_visible(False)

    bar_plot_f.savefig(f'CDN_interneuropil_inputs_to_{cell_id}_barplot.svg')

In [ ]:
# Plot all interneuropil neurons
inter_n = cdn_in[upstream_c2_grouped['output_type'] == 'interneuron']
inter_n['weight'] = upstream_c2_grouped.loc[inter_n.index.values]['weight']
inter_n['bodyId'] = inter_n.index.values
inter_n.sort_values(by='weight', ascending=False, inplace=True)

max_val = inter_n['weight'].max()

inter_n[['weight', 'bodyId', 'ntGlutamateProb', 'ntAcetylcholineProb', 'ntGabaProb', 'type', 'pre', 'post']]

# all_rois = [c.fetch_roi_mesh(roiname) for roiname in ['ANm', 'LegNp(T1)(L)',
#  'LegNp(T1)(R)',
#  'LegNp(T2)(L)',
#  'LegNp(T2)(R)',
#  'LegNp(T3)(L)',
#  'LegNp(T3)(R)',]
# ]

grid = gridplot([
     [
          plot_neurons(inter_n, idx*8 + sub_idx, 1, color = '#58479E')
          for sub_idx in range(8)
     ]
     for idx in range(3)
])

show(grid)
export_png(grid, filename = 'Top_CDN_interneuron_inputs.png')


In [ ]:
inter_n_main = inter_n.iloc[:24]
inter_n_main['entropy'] = -(inter_n_main['ntGlutamateProb']*np.log2(inter_n_main['ntGlutamateProb']) + inter_n_main['ntAcetylcholineProb']*np.log2(inter_n_main['ntAcetylcholineProb']) + inter_n_main['ntGabaProb']*np.log2(inter_n_main['ntGabaProb']))
plt.plot(
    inter_n_main['weight'],
    inter_n_main['entropy'],
    'o',
    color = '#58479E',
)

plt.gca().set_ylabel('Entropy of synapse classification')
plt.gca().set_xlabel('Synapses onto CDNs')

plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

plt.gca().set_yticks([0,0.569,1.18, -np.log2(1/3)], labels = ['Total\ncertainty','90%\ncertainty','70%\ncertainty','Total\nuncertainty'])
plt.gca().set_ylim(0,-np.log2(1/3))

plt.gca().set_xlim(0, 300)
plt.gca().set_xticks([0, 150, 300])
plt.gcf().set_size_inches(2,2)

plt.savefig('CDN_interneuron_inputs_entropy.svg')


In [ ]:
inter_n.columns

In [ ]:
_, cdn_to_inter_inputs_conn = fetch_adjacencies(
    sources = NC(bodyId = cluster2),
    targets = NC(bodyId=inter_n.index.values)
)

inter_n_recip = inter_n.merge(
    cdn_to_inter_inputs_conn.groupby('bodyId_post').agg({'weight' : 'sum'}).sort_values(by='weight', ascending=False),
    left_index=True,
    right_index=True,
    suffixes=['_onto_cdns', '_from_cdns']
)[['weight_onto_cdns', 'weight_from_cdns', 'entropy', 'predictedNt']]



plt.scatter(
    inter_n_recip['weight_onto_cdns'][inter_n_recip['predictedNt'] == 'acetylcholine'],
    inter_n_recip['weight_from_cdns'][inter_n_recip['predictedNt'] == 'acetylcholine'],
    color = SHAMROCK,
    alpha = 1-inter_n_recip['entropy']/np.log2(3),
    linewidth = 0,
)

plt.gca().scatter(
    inter_n_recip['weight_onto_cdns'][inter_n_recip['predictedNt'] == 'glutamate'],
    inter_n_recip['weight_from_cdns'][inter_n_recip['predictedNt'] == 'glutamate'],
    color = BYZANTINE,
    alpha = 1-inter_n_recip['entropy']/np.log2(3),
    linewidth = 0,
)

plt.gca().scatter(
    inter_n_recip['weight_onto_cdns'][inter_n_recip['predictedNt'] == 'gaba'],
    inter_n_recip['weight_from_cdns'][inter_n_recip['predictedNt'] == 'gaba'],
    color = LAPIS,
    alpha = 1-inter_n_recip['entropy']/np.log2(3),
    linewidth = 0,
)

plt.gca().set_xlabel('Synapses onto CDNs')

plt.gca().set_ylabel('Synapses from CDNs')

plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

plt.gca().set_xticks([0, 150, 300])
plt.gca().set_yticks([0, 200, 400])

plt.gcf().set_size_inches(2,2)

plt.savefig('CDN_interneuron_inputs_by_type.svg')

In [ ]:
inter_n_recip['entropy']/np.log2(3)

In [ ]:
# Scatter plot of intern

_, intern_from_cdn_conn = fetch_adjacencies(
    sources = NC(
        bodyId = cluster2
    ),
    targets = NC(
        bodyId = intern['bodyId'].values
    )
)

_, intern_to_cdn_conn = fetch_adjacencies(
    sources = NC(
        bodyId = intern['bodyId'].values
    ),
    targets = NC(
        bodyId = cluster2
    )
)

mutual_conn = intern_from_cdn_conn.merge(
    intern_to_cdn_conn,
    left_on='bodyId_post',
    right_on='bodyId_pre',
    suffixes=('_in', '_out')
).groupby(
    'bodyId_post_in'
).agg(
    {
        'weight_in' : 'sum',
        'weight_out' : 'sum'
    }
)

mutual_conn = mutual_conn[mutual_conn['weight_in'] > 90]

mutual_df = fetch_neurons(
    NC(bodyId = mutual_conn.index.values)
)[0]

mutual_df = mutual_df.set_index('bodyId')

mutual_conn['neurotransmitter'] = mutual_df.loc[mutual_conn.index.values]['predictedNt']
mutual_conn['entropy'] = mutual_df.loc[mutual_conn.index.values][['ntAcetylcholineProb', 'ntGabaProb', 'ntGlutamateProb']].apply(lambda x: -np.sum(x*np.log2(x)), axis=1)

plt.scatter(
    mutual_conn[mutual_conn['neurotransmitter'] == 'gaba']['weight_in'],
    mutual_conn[mutual_conn['neurotransmitter'] == 'gaba']['weight_out'],
    color = LAPIS,
    alpha = 1+mutual_conn[mutual_conn['neurotransmitter'] == 'gaba']['entropy'].values/np.log2(1/3),
    edgecolors=None,
    linewidths = 0,
)

plt.scatter(
    mutual_conn[mutual_conn['neurotransmitter'] == 'acetylcholine']['weight_in'],
    mutual_conn[mutual_conn['neurotransmitter'] == 'acetylcholine']['weight_out'],
    color = BYZANTINE,
    alpha = 1+mutual_conn[mutual_conn['neurotransmitter'] == 'acetylcholine']['entropy'].values/np.log2(1/3),
    edgecolors=None,
    linewidths = 0,
)

plt.scatter(
    mutual_conn[mutual_conn['neurotransmitter'] == 'glutamate']['weight_in'],
    mutual_conn[mutual_conn['neurotransmitter'] == 'glutamate']['weight_out'],
    color = SHAMROCK,
    alpha = 1+mutual_conn[mutual_conn['neurotransmitter'] == 'glutamate']['entropy'].values/np.log2(1/3),
    edgecolors=None,
    linewidths = 0,
)

plt.xlabel('Synapses received from CDNs')
plt.ylabel('Synapses onto CDNs')

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().set_aspect('equal')
plt.gca().set_xlim(0,2200)
plt.gca().set_ylim(0,2200)
plt.gca().set_xticks([0,1000,2000])
plt.gca().set_yticks([0,1000,2000])
plt.gcf().set_size_inches(2,2)

plt.gcf().savefig('CDN_interneuron_mutual_synapses.svg')

In [ ]:
mutual_conn[mutual_conn['neurotransmitter'] == 'gaba'].iloc[:20]



In [ ]:
types = ['descending', 'interneuropil', 'interneuron']
fraction_of_inputs = {
    TYPE : 
        downstream_c2_grouped.loc[downstream_c2_grouped[downstream_c2_grouped['output_type'] == TYPE].index.values]['weight'].values/(
        fetch_neurons(NC(bodyId=downstream_c2_grouped[downstream_c2_grouped['output_type'] == TYPE].index.values))[0]['post'].values
        )
    for TYPE in types
}

In [ ]:
from scipy.stats import gaussian_kde

# gk = gaussian_kde(np.exp(fraction_of_inputs['descending']))

# plt.plot(
#     np.logspace(-4, 0, 100),
#     gk(np.exp(np.logspace(-4, 0, 100))),
#     color = SHAMROCK,
#     label = 'Descending'
# )

# plt.plot(
#     np.logspace(-4, 0, 100),
#     gaussian_kde(np.exp(fraction_of_inputs['interneuropil']))(np.exp(np.logspace(-4, 0, 100))),
#     color = BYZANTINE,
#     label = 'Interneuropil'
# )

plt.hist(
    fraction_of_inputs['descending'],
    bins = np.logspace(-4, 0, 20),
    color = LAPIS,
    #alpha = 0.3,
    stacked=True,
    label = 'Descending',
    histtype='step',
    linewidth = 3
)

plt.hist(
    fraction_of_inputs['interneuropil'],
    bins = np.logspace(-4, 0, 20),
    color = SHAMROCK,
    #alpha = 0.3,
    stacked=True,
    label = 'Interneuropil',
    histtype='step',
    linewidth = 3
)

plt.hist(
    fraction_of_inputs['interneuron'],
    bins = np.logspace(-4, 0, 20),
    color = '#58479E',
    #alpha = 0.3,
    stacked = True,
    label = 'Interneuron',
    histtype='step',
    linewidth = 3
)

plt.gca().set_ylabel('Number of neurons\nof class')
plt.gca().set_xlabel('Fraction of input synapses\ncoming from CDNs')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().set_ylim(1, 1e2)
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')

plt.gcf().set_size_inches(3,1.5)
plt.gcf().savefig('CDN_fraction_of_inputs_to_output_cells_by_class.svg')
